In [18]:
# Import required libraries 
import sqlite3 
import pandas as pd 
  
# Connect to SQLite database 
conn = sqlite3.connect(r'FlightBooking.db') 
  
# # Load CSV data into Pandas DataFrame 
eval_df = pd.read_csv('evals/policyEvals.csv')
# # Write the data to a sqlite table 
eval_df.to_sql('bookinginfo', conn, if_exists='replace', index=False) 

17

In [19]:
cur = conn.cursor() 
# Fetch and display result 
for row in cur.execute('SELECT * FROM bookinginfo'): 
    print(row) 

(0, 'I want to cancel my flight as I am initiating the cancellation myself.', 'I would like to cancel my flight, please.', 'John Doe', 'ABC123', 'XYZ789', 'process_partial_refund', '{"booking_reference":"ABC123","refund_amount":198.0}', 'refundable', 'partial_refund', None, 198.0)
(1, 'I want to change my flight to a different date.', "I'd like to initiate a change to my flight for a later date.", 'Jane Smith', 'DEF456', 'XYZ123', 'apply_change_fee', '{"booking_reference":"DEF456"}', 'nonrefundable', None, None, None)
(2, 'I want to request a refund because my flight was canceled by the airline due to weather.', 'My flight was canceled by the airline, and I want a refund.', 'Emma Johnson', 'GHI789', 'ABC123', 'process_full_refund', '{"booking_reference":"GHI789"}', 'refundable', 'full_refund', None, None)
(3, 'I want to cancel my flight but I think it is non-refundable.', 'I want to cancel my non-refundable ticket.', 'Michael Brown', 'JKL012', 'DEF456', 'offer_flight_credit', '{"bookin

In [5]:
columns = conn.execute(f"PRAGMA table_info('bookinginfo');").fetchall()

In [6]:
columns

[(0, 'ID', 'INTEGER', 0, None, 0),
 (1, 'context', 'TEXT', 0, None, 0),
 (2, 'request', 'TEXT', 0, None, 0),
 (3, 'full_name', 'TEXT', 0, None, 0),
 (4, 'booking_ref', 'TEXT', 0, None, 0),
 (5, 'flight_no', 'TEXT', 0, None, 0),
 (6, 'expected_function', 'TEXT', 0, None, 0),
 (7, 'expected_inputs', 'TEXT', 0, None, 0),
 (8, 'ticket_type', 'TEXT', 0, None, 0),
 (9, 'fare_rules', 'TEXT', 0, None, 0),
 (10, 'medical_certificate', 'TEXT', 0, None, 0),
 (11, 'refund_amount', 'REAL', 0, None, 0)]

In [9]:
tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

In [13]:
def get_table_names(conn):
    """Return a list of table names."""
    table_names = []
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names

def get_column_names(conn, table_name):
    """Return a list of column names."""
    column_names = []
    columns = conn.execute(f"PRAGMA table_info('{table_name}');").fetchall()
    for col in columns:
        column_names.append(col[1])
    return column_names

def get_database_info(conn):
    """Return a list of dicts containing the table name and columns for each table in the database."""
    table_dicts = []
    for table_name in get_table_names(conn):
        columns_names = get_column_names(conn, table_name)
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts

In [15]:
database_schema_dict = get_database_info(conn)
database_schema_string = "\n".join(
    [
        f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)

In [16]:
database_schema_string

'Table: bookinginfo\nColumns: ID, context, request, full_name, booking_ref, flight_no, expected_function, expected_inputs, ticket_type, fare_rules, medical_certificate, refund_amount'

In [17]:
conn.close()